네이버 로그인 API 

https://developers.naver.com

- application >> 애플리케이션 등록(api 이용신청)
- 애플리케이션 이름 : naver_api 
- 사용 api : 검색
- 비로그인 오픈 api 서비스 환경 : android 설정 >> com.example.naver_api

In [1]:
import re
import json
import math
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd

Naver Developers에서 애플리케이션 생성 및 api 신청 및 가져오기

In [8]:
naver_client_id = "_kLh6jGjY0cv3OkcX8rI"
naver_client_secret = "ssjIp9J4gX"

네이버 블로그 내용 가져오기

In [9]:
# 블로그 전체 개수 가져오는 함수 만들기 

def get_blog_count(query, display):
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    # url 에서 query 부분 어떤 검색 키워드 넣는지에 따라 결과 반환 
    request = urllib.request.Request(search_url)
    
    # header 에 개발자 발급 받은 client id, client secret 입력 
    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request) # 실제 url 해당 결과 요청
    response_code = response.getcode()         # 반환되는 code 확인 

    if response_code == 200:       # respond code == 200 : 서버가 요청을 제대로 처리했다는 의미 
        response_body = response.read()  # 요청한 페이지 읽어오기 
        response_body_dict = json.loads(response_body.decode('utf-8'))  # json.loads: json 파일 읽어오기 

        print("Last build date: " + str(response_body_dict['lastBuildDate']))
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000: 
                blog_count = 1000   # 네이버 검색은 최대 1000개 검색결과만 보여주기 때문 최대값은 1000
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))

        return blog_count

# get_blog_count 함수 사용, 실제 수집한 블로그 개수 확인 

In [18]:
# 검색된 블로그 내용 가져오기 위한 함수 생성 

def get_blog_post(query, display, start_index, sort):
    global no, df
    
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" \   # \: 줄바꿈
    + encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort
    
    '''
    - display : 검색 결과 출력 건수 지정 (기본값 : 10, 최대 100개)
    - start_index: 검색 시작 위치로 1부터 최대 100
    - sort : 정렬 옵션 
      - sim : 유사도순, date: 날짜 순 
    '''
    
    
    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code == 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:
                remove_html_tag = re.compile('<.*?>') 
                # re.compile : html tag 제거 
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                # post_date : 날짜 형식으로 변환 

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')
                 
                # txt 파일로 저장     
                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("#", end='')
                
            except:
                item_index += 1

In [12]:
no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = "스마트폰"   # 검색을 원하는 문자열로서 UTF-8로 인코딩한다.
display = 10           # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 1000까지 가능
sort = "sim"           # 정렬 옵션: sim(유사도순, 기본값), date(날짜순)

# 블로그를 DataFrame에 저장
df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):
    get_blog_post(query, display, start_index, sort)



Last build date: Sun, 13 Mar 2022 23:39:27 +0900
Total: 7104919
Start: 1
Display: 10
Blog total: 710492
Blog count: 1000
###############################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [13]:
df

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
1,ZEM 초등학생 스마트폰 관리앱으로 걱정 덜어요,https://blog.naver.com/rladbrud8791?Redirect=L...,사실 제가 최근까지도 고민하고 있었던 부분이 바로 아이의 스마트폰 필요성 때문인데요...,갱이랑 정이랑♡,https://blog.naver.com/rladbrud8791,20220226,\n\n\n\n\n\n\n리뷰\n\n\n\n\nZEM 초등학생 스마트폰 관리앱으로 ...
2,아이폰 SE3 사전예약 출시일 가격 정보! 애플의 보급형 스마트폰,https://blog.naver.com/hongganz?Redirect=Log&l...,애플의 보급형 스마트폰 아이폰 SE3 가 출시한다. 보통 대부분의 스마트폰들은 급나...,이웃삼촌이 들려주는 IT 이야기,https://blog.naver.com/hongganz,20220313,\n\n\n\n\n\n\n\n\n\n출시 정보\n\n\n\n\n아이폰 SE3 사전예...
3,왜 다들 스마트폰 삼각대 추천 하는지 궁금해서 써본 후기,https://blog.naver.com/tnwpqla?Redirect=Log&lo...,하다가 스마트폰 삼각대 를 구매를 해야겠단 생각을 해서 제대로 된 것을 알아보려고 ...,초컬렛::대학생 블로그,https://blog.naver.com/tnwpqla,20220310,\n\n\n\n\n\n\n협찬+포스팅알바\n\n\n\n\n왜 다들 스마트폰 삼각대 ...
4,스마트폰 백업 필요하다면 네이버 마이박스(MY BOX)로!,https://blog.naver.com/dogslife78?Redirect=Log...,최근 컴퓨터나 노트북 뿐만 아니라 항상 소지하고 다니는 스마트폰이나 태블릿과 같은 ...,노숙페이스,https://blog.naver.com/dogslife78,20220301,"\n\n\n\n\n\n\n가전, 그리고 IT\n\n\n\n\n스마트폰 백업 필요하다..."
5,"맥갤럭시 스마트폰 연결, 백업",https://blog.naver.com/yhj5738?Redirect=Log&lo...,스마트폰에 스마트 스위치 앱이 설치되지 않을 경우에는 맥에서 Smaert Switc...,Dream girl's life,https://blog.naver.com/yhj5738,20220304,"\n\n\n\n\n\n\n관리방법\n\n\n\n\n맥갤럭시 스마트폰 연결, 백업 \..."
...,...,...,...,...,...,...,...
996,스마트폰이 떨어질 땐 휘어진다?,http://news.egloos.com/4161002,일본 후지쯔에서 만든 애로우즈 비-라는 스마트폰 브랜드가 있습니다. 떨어져도 잘 깨...,"자그니 블로그 : 거리로 나가자, 키스를 하자",http://news.egloos.com/,20181009,
997,"부품 교체 및 업글 가능한 스마트폰, 페어폰 3 발매",http://news.egloos.com/4175365,모듈화된 스마트폰 프로젝트 '아라'를 진행한 적이 있습니다. 물론 제대로 된 시제품...,"자그니 블로그 : 거리로 나가자, 키스를 하자",http://news.egloos.com/,20190909,
998,"MWC2018, 최고와 최악의 스마트폰은 무엇?",https://enjoiyourlife.com/2436?category=581147,상반기 스마트폰 시장에서 어떤 성과를 보여줄 것인지에 관심이 모아지고 있습니다. 특...,HOME,http://enjoiyourlife.com/,20180302,
999,유튜브 동영상 보기 좋은 스마트폰 리스트 발표,https://photohistory.tistory.com/18077,DRM (디지털 저작권 관리)의 성능이 높은 폰 유튜브는 이 6가지 항목을 토대로 ...,사진은 권력이다,https://photohistory.tistory.com/,20180810,


In [14]:
df.to_csv("./smartphone.csv", header=True, index=False)